# TigerGraph AMLSim Demo

To Get Started you will need an instance of TigerGraph running. The fastest way to get a box running is using **https://tgcloud.io**. 

If it's your first time using the cloud portal checkout [**Getting Started with TigerGraph 3.0**](https://www.tigergraph.com/blog/getting-started-with-tigergraph-3-0/)

>**Note:** This notebook will work if you have an existing solution, by creating a new graph, but it's suggested that you have a blank instance. At the bottom of the notebook you will see a commented out cell that will allow you to clear your existing solution. Use that command wisely as it deletes everything on your existing solution. 

## About AMLSim
The AMLSim project is intended to provide a multi-agent based simulator that generates synthetic banking transaction data together with a set of known money laundering patterns - mainly for the purpose of testing machine learning models and graph algorithms.

**AMLSim Official GitHub**
https://github.com/IBM/AMLSim

**AMLSim WIKI** https://github.com/IBM/AMLSim/wiki

**AMLSim Data** https://github.com/IBM/AMLSim/wiki/Download-Example-Data-Set

## Setup TigerGraph Server

> Note: If this is your first time using TigerGraph Cloud, checkout the this [blog](https://www.tigergraph.com/blog/getting-started-with-tigergraph-3-0/) to learn the basics of deploying an instance on tgcloud.io

For this workshop you can choose `Blank v3` as the starter kit.

![MR](https://miro.medium.com/max/1400/1*8ZGz7MccIpFYSaqO1bszyQ.png)

Once you have the box provisioned with the Starter Kit mentioned above open GraphStudio.

![GS](https://miro.medium.com/max/1400/1*D9Ya_eBWstx-xv_i-9gbQw.png)

On the top right you will see `Global View`. Click on that and choose `MyGraph`. Once you select `MyGraph`, Global View will go away and it will look simliar to this.

![GS2](https://miro.medium.com/max/1400/1*d6hPhuJE7qttBhCk37cTWA.png)

Perfect. Now you Graph is UP. Let's move on. 

### Grab and Import Packages

In [ ]:
# FETCH PACKAGES
!pip install pyTigerGraphBeta
import pyTigerGraphBeta as tg
import pandas as pd

### Setup Connection to TigerGraph

In [96]:
# SETUP TIGERGRAPH CONNECTION
conn = tg.TigerGraphConnection(host="https://aml-paysim.i.tgcloud.io", username="tigergraph", password="tigergraph")

### Fetch Data From GitHub Repo

In [ ]:
# GRAB DATA FILES 
!git clone https://github.com/TigerGraph-OSS/AMLSim_Python_Lab

### Loading DataFrames


In [5]:
accounts = pd.read_csv("/content/AMLSim_Python_Lab/accounts.csv", sep=",")
alerts = pd.read_csv("/content/AMLSim_Python_Lab/alerts.csv", sep=",")
transactions = pd.read_csv("/content/AMLSim_Python_Lab/transactions.csv", sep=",")

## Create Graph

### Create Global Vertices and Edges

In [ ]:
# DEFINE / CREATE ALL EDGES AND VERTICES 
print(conn.gsql('''
CREATE VERTEX Country (PRIMARY_ID id STRING) WITH primary_id_as_attribute="true"
CREATE VERTEX Customer (PRIMARY_ID id STRING) WITH primary_id_as_attribute="true"
CREATE VERTEX Account (PRIMARY_ID id STRING, init_balance DOUBLE, account_type STRING, tx_behavior INT, pagerank FLOAT, label INT, current_balance DOUBLE, min_send_tx DOUBLE, min_recieve_tx DOUBLE, max_send_tx DOUBLE, max_recieve_tx DOUBLE, avg_send_tx DOUBLE, avg_recieve_tx DOUBLE, cnt_recieve_tx INT, cnt_send_tx INT) WITH primary_id_as_attribute="true"
CREATE VERTEX Transaction (PRIMARY_ID id STRING, tx_behavior_id INT, amount DOUBLE, is_fraud BOOL) WITH primary_id_as_attribute="true"
CREATE VERTEX Alert (PRIMARY_ID id STRING, alert_type STRING, ts INT) WITH primary_id_as_attribute="true"
CREATE UNDIRECTED EDGE Based_In (From Customer, To Country)
CREATE UNDIRECTED EDGE Customer_Account (From Customer, To Account)
CREATE UNDIRECTED EDGE Transaction_Flagged (From Transaction, To Alert)
CREATE DIRECTED EDGE Send_To (From Account, To Account) WITH REVERSE_EDGE="reverse_Send_To"
CREATE DIRECTED EDGE Send_Transaction (From Account, To Transaction, ts INT, tx_type STRING) WITH REVERSE_EDGE="reverse_Send_Transaction"
CREATE DIRECTED EDGE Recieve_Transaction (From Transaction, To Account, ts INT, tx_type STRING) WITH REVERSE_EDGE="reverse_Recieve_Transaction"
''', options=[]))


### Create AMLSim Graph

In [77]:
# CREATE GRAPH "AMLSim"
print(conn.gsql('''CREATE GRAPH AMLSim(Country, Customer, Account, Transaction, Alert, Based_In, Customer_Account, Transaction_Flagged, Send_To, Send_Transaction, reverse_Send_Transaction, Recieve_Transaction, reverse_Recieve_Transaction)''', options=[]))

['Stopping GPE GSE RESTPP', 'Successfully stopped GPE GSE RESTPP in 0.006 seconds', 'Starting GPE GSE RESTPP', 'Successfully started GPE GSE RESTPP in 0.097 seconds', 'The graph AMLSim is created.']


In [ ]:
# TEST TO SEE IF GRAPH SCHEMA BUILT
print(conn.gsql('''LS''', options=[]))

### Set New Graph Parameters

In [98]:
# SET NEW GRAPH PARAMS
# Instructions on generating a secret https://towardsdatascience.com/generating-a-secret-in-tigergraph-e5139d52dff6
conn.graphname = "AMLSim"
conn.apiToken = conn.getToken("437b3u5grimgkv9qhk7vu819me6hq3ge")
# conn.apiToken = conn.getToken(conn.createSecret())

In [ ]:
# ACCOUNTS DATAFRAME
accounts['ACCOUNT_ID'] = accounts['ACCOUNT_ID'].astype(str)
accounts.head()

In [ ]:
# ALERTS DATAFRAME
alerts['ALERT_ID'] = alerts['ALERT_ID'].astype(str)
alerts['SENDER_ACCOUNT_ID'] = alerts['SENDER_ACCOUNT_ID'].astype(str)
alerts['RECEIVER_ACCOUNT_ID'] = alerts['RECEIVER_ACCOUNT_ID'].astype(str)
alerts.head()

## Load Data

In [ ]:
# TRANSACTION DATAFRAME
transactions['TX_ID'] = transactions['TX_ID'].astype(str)
transactions['SENDER_ACCOUNT_ID'] = transactions['SENDER_ACCOUNT_ID'].astype(str)
transactions['RECEIVER_ACCOUNT_ID'] = transactions['RECEIVER_ACCOUNT_ID'].astype(str)
transactions.head()

In [ ]:
# UPSERT VERTEX "Customer"
v_customer = conn.upsertVertexDataFrame(accounts, "Customer", "CUSTOMER_ID", attributes={"id": "CUSTOMER_ID"})
print(str(v_customer) + " Customer VERTICES Upserted")

# UPSERT VERTEX "Account"
v_account = conn.upsertVertexDataFrame(accounts, "Account", "ACCOUNT_ID", attributes={"id": "ACCOUNT_ID", "init_balance": "INIT_BALANCE", "account_type": "ACCOUNT_TYPE", "tx_behavior": "TX_BEHAVIOR_ID"})
print(str(v_account) + " Account VERTICES Upserted")

# UPSERT VERTEX "Transaction"
v_transaction = conn.upsertVertexDataFrame(transactions, "Transaction", "TX_ID", attributes={"id": "TX_ID", "amount": "TX_AMOUNT", "is_fraud": "IS_FRAUD"})
print(str(v_transaction) + " Transaction VERTICES Upserted")

# UPSERT VERTEX "Alert"
v_alert = conn.upsertVertexDataFrame(alerts, "Alert", "ALERT_ID", attributes={"id": "ALERT_ID", "alert_type": "ALERT_TYPE", "ts": "TIMESTAMP"})
print(str(v_alert) + " Alert VERTICES Upserted")

# UPSERT EDGE "Send_Transaction"
e_send_transaction = conn.upsertEdgeDataFrame(transactions, "Account", "Send_Transaction", "Transaction", from_id="SENDER_ACCOUNT_ID", to_id="TX_ID", attributes={"ts": "TIMESTAMP", "tx_type": "TX_TYPE"})
print(str(e_send_transaction) + " Send_Transaction EDGES Upserted")

# UPSERT EDGE "Send_Transaction"
e_recieve_transaction = conn.upsertEdgeDataFrame(transactions, "Transaction", "Recieve_Transaction", "Account", from_id="TX_ID", to_id="RECEIVER_ACCOUNT_ID", attributes={"ts": "TIMESTAMP", "tx_type": "TX_TYPE"})
print(str(e_recieve_transaction) + " Recieve_Transaction EDGES Upserted")

# UPSERT EDGE "Send_To"
e_send_to = conn.upsertEdgeDataFrame(transactions, "Account", "Send_To", "Account", from_id="SENDER_ACCOUNT_ID", to_id="RECEIVER_ACCOUNT_ID", attributes={})
print(str(e_send_to) + " Send_To EDGES Upserted")

# UPSERT EDGE "Transaction_Flagged"
e_transaction_flagged = conn.upsertEdgeDataFrame(alerts, "Transaction", "Transaction_Flagged", "Alert", from_id="TX_ID", to_id="ALERT_ID", attributes={})
print(str(e_transaction_flagged) + " Transaction_Flagged EDGES Upserted")

# UPSERT EDGE "Customer_Account"
e_customer_account = conn.upsertEdgeDataFrame(accounts, "Customer", "Customer_Account", "Account", from_id="CUSTOMER_ID", to_id="ACCOUNT_ID", attributes={})
print(str(e_customer_account) + " Customer_Account EDGES Upserted")

# UPSERT EDGE "Based_In"
e_based_in = conn.upsertEdgeDataFrame(accounts, "Customer", "Based_In", "Country", from_id="CUSTOMER_ID", to_id="COUNTRY", attributes={})
print(str(e_based_in) + " Based_In EDGES Upserted")

# PRINT OUT STATS
print("=====TOTAL_UPSERTS=====")
print(str(v_customer+v_account+v_transaction+v_alert) + " TOTAL VERTICES")
print(str(e_send_transaction+e_recieve_transaction+e_send_to+e_transaction_flagged+e_customer_account+e_based_in) + " TOTAL EDGES")


## Features Generation for Machine Learning

In [ ]:
# CREATE QUERY FOR GENERATING FEATURES ABOUT ACCOUNT ACTIVITY
print(conn.gsql('''
CREATE QUERY accountActivity() FOR GRAPH AMLSim { 
  
  SumAccum<DOUBLE> @s_sumAmt, @r_sumAmt;
  SumAccum<DOUBLE> @s_txCnt, @r_txCnt;
  MinAccum<DOUBLE> @s_minAmt, @r_minAmt;
  MaxAccum<DOUBLE> @s_maxAmt, @r_maxAmt;
  AvgAccum @s_avgAmt, @r_avgAmt;

  
  
  Seed = {Account.*};
  
  acctSend = SELECT tgt FROM Seed:s -(Send_Transaction:e)-> Transaction:tgt
             ACCUM s.@s_sumAmt += tgt.amount, 
                   s.@s_txCnt += 1,
                   s.@s_minAmt += tgt.amount, 
                   s.@s_maxAmt += tgt.amount,
                   s.@s_avgAmt += tgt.amount
            POST-ACCUM
                s.current_balance = s.@s_sumAmt - s.init_balance,
                s.min_send_tx = s.@s_minAmt,
                s.max_send_tx = s.@s_maxAmt,
                s.avg_send_tx = s.@s_avgAmt,
                s.cnt_send_tx = s.@s_txCnt;
                

  
  acctRecieve = SELECT tgt FROM Seed:s -(reverse_Recieve_Transaction:e)-> Transaction:tgt
                ACCUM s.@r_sumAmt += tgt.amount, 
                      s.@r_txCnt += 1,
                      s.@r_minAmt += tgt.amount, 
                      s.@r_maxAmt += tgt.amount,
                      s.@r_avgAmt += tgt.amount
                      
                POST-ACCUM
                  s.current_balance = s.@r_sumAmt + s.init_balance,
                  s.min_recieve_tx = s.@r_minAmt,
                  s.max_recieve_tx = s.@r_maxAmt,
                  s.avg_recieve_tx = s.@r_avgAmt,
                  s.cnt_recieve_tx = s.@r_txCnt;
              
  PRINT "Features Have Been Calculated";

}
''', options=[]))

In [ ]:
# INSTALL QUERY FOR GENERATING FEATURES ABOUT ACCOUNT ACTIVITY
print(conn.gsql('''INSTALL QUERY accountActivity''', options=[]))

## Queries to Install for Feature Generation

In [ ]:
# CREATE QUERY FOR GENERATING FEATURES USING LABEL PROP
print(conn.gsql('''
CREATE QUERY label_prop (SET<STRING> v_type, SET<STRING> e_type, INT max_iter, INT output_limit, BOOL print_accum = TRUE, STRING file_path = "", STRING attr = "") FOR GRAPH AMLSim{
# Partition the vertices into communities, according to the Label Propagation method.
# Indicate community membership by assigning each vertex a community ID.

        OrAccum @@changed = true;
        MapAccum<INT, INT> @map;     # <communityId, numNeighbors>
        MapAccum<INT, INT> @@commSizes;   # <communityId, members>
        SumAccum<INT> @label, @num;  
        FILE f (file_path);
        Start = {v_type};

# Assign unique labels to each vertex
        Start = SELECT s FROM Start:s ACCUM s.@label = getvid(s);

# Propagate labels to neighbors until labels converge or the max iterations is reached
        WHILE @@changed == true LIMIT max_iter DO
                @@changed = false;
                Start = SELECT s 
                        FROM Start:s -(e_type:e)-> :t
                        ACCUM t.@map += (s.@label -> 1)  # count the occurrences of neighbor's labels
                        POST-ACCUM
                                INT maxV = 0,
                                INT label = 0,
                                # Iterate over the map to get the neighbor label that occurs most often
                                FOREACH (k,v) IN t.@map DO
                                        CASE WHEN v > maxV THEN
                                                maxV = v,
                                                label = k
                                        END
                                END,
                                # When the neighbor search finds a label AND it is a new label
                                # AND the label's count has increased, update the label.
                                CASE WHEN label != 0 AND t.@label != label AND maxV > t.@num THEN
                                        @@changed += true,
                                        t.@label = label,
                                        t.@num = maxV
                                END,
                                t.@map.clear();
        END;

        Start = {v_type};
        Start =  SELECT s FROM Start:s
                  POST-ACCUM 
                        IF attr != "" THEN s.setAttr(attr, s.@label) END,
                        IF file_path != "" THEN f.println(s, s.@label) END,
                        IF print_accum THEN @@commSizes += (s.@label -> 1) END
                  LIMIT output_limit;

        IF print_accum THEN 
           PRINT @@commSizes;
           PRINT Start[Start.@label];
        END;
}
''', options=[]))

In [ ]:
# INSTALL QUERY FOR GENERATING FEATURES USING LABEL PROP
print(conn.gsql('''INSTALL QUERY label_prop''', options=[]))

In [ ]:
# CREATE QUERY FOR GENERATING FEATURES USING PAGERANK
print(conn.gsql('''
CREATE QUERY pageRank (STRING v_type, STRING e_type,
 FLOAT max_change=0.001, INT max_iter=25, FLOAT damping=0.85, INT top_k = 100,
 BOOL print_accum = TRUE, STRING result_attr =  "", STRING file_path = "",
 BOOL display_edges = FALSE) FOR GRAPH AMLSim{
/*
 Compute the pageRank score for each vertex in the GRAPH
 In each iteration, compute a score for each vertex:
     score = (1-damping) + damping*sum(received scores FROM its neighbors).
 The pageRank algorithm stops when either of the following is true:
 a) it reaches max_iter iterations;
 b) the max score change for any vertex compared to the last iteration <= max_change.
 v_type: vertex types to traverse          print_accum: print JSON output
 e_type: edge types to traverse            result_attr: INT attr to store results to
 max_iter; max #iterations                 file_path: file to write CSV output to
 top_k: #top scores to output              display_edges: output edges for visualization
 max_change: max allowed change between iterations to achieve convergence
 damping: importance of traversal vs. random teleport

 This query supports only taking in a single edge for the time being (8/13/2020).
*/
	TYPEDEF TUPLE<VERTEX Vertex_ID, FLOAT score> Vertex_Score;
	HeapAccum<Vertex_Score>(top_k, score DESC) @@topScores;
	MaxAccum<FLOAT> @@max_diff = 9999;    # max score change in an iteration
	SumAccum<FLOAT> @recvd_score = 0; # sum of scores each vertex receives FROM neighbors
	SumAccum<FLOAT> @score = 1;           # initial score for every vertex is 1.
	SetAccum<EDGE> @@edgeSet;             # list of all edges, if display is needed
	FILE f (file_path);

# PageRank iterations	
	Start = {v_type};                     # Start with all vertices of specified type(s)
	WHILE @@max_diff > max_change LIMIT max_iter DO
			@@max_diff = 0;
			V = SELECT s
				FROM Start:s -(e_type:e)-> v_type:t
				ACCUM t.@recvd_score += s.@score/(s.outdegree(e_type)) 
				POST-ACCUM s.@score = (1.0-damping) + damping * s.@recvd_score,
						   s.@recvd_score = 0,
						   @@max_diff += abs(s.@score - s.@score');
	END; # END WHILE loop

# Output
	IF file_path != "" THEN
	  f.println("Vertex_ID", "PageRank");
	END;

	V = SELECT s FROM Start:s
		POST-ACCUM 
			IF result_attr != "" THEN s.setAttr(result_attr, s.@score) END,
			IF file_path != "" THEN f.println(s, s.@score) END,
			IF print_accum THEN @@topScores += Vertex_Score(s, s.@score) END;

	IF print_accum THEN
		PRINT @@topScores;
		IF display_edges THEN
			PRINT Start[Start.@score];
			Start = SELECT s
					FROM Start:s -(e_type:e)-> v_type:t
					ACCUM @@edgeSet += e;
		   PRINT @@edgeSet;
		END;
	END;
}
''', options=[]))

In [ ]:
# INSTALL QUERY FOR GENERATING FEATURES USING PAGERANK
print(conn.gsql('''INSTALL QUERY pageRank''', options=[]))


## Queries for GraphStudio
Select Account Query
```
CREATE QUERY selectAccount(STRING acct) FOR GRAPH AMLSim { 
  seed = {Account.*};
  S1 = SELECT s FROM seed:s WHERE s.id == acct; 
  PRINT S1; 
}
```
Select Account Query
```
CREATE QUERY selectTopPageRank() FOR GRAPH AMLSim { 
  seed = {Account.*};
  S1 = SELECT s FROM seed:s ORDER BY s.pagerank DESC LIMIT 10;
  PRINT S1; 
}
```
Select Account Transactions
```
CREATE QUERY selectAccountTx(STRING acct) FOR GRAPH AMLSim { 
  ListAccum<EDGE> @@txSend, @@txRecieve;
  seed = {Account.*};
  
  SendTx = SELECT tgt FROM seed:s -(Send_Transaction:e)-> Transaction:tgt
           WHERE s.id == acct
           ACCUM @@txSend +=  e;
  
  RecieveTx = SELECT tgt FROM seed:s -(reverse_Recieve_Transaction:e)-> Transaction:tgt
              WHERE s.id == acct
              ACCUM @@txRecieve += e;
  
  PRINT @@txSend, @@txRecieve; 
}
```

In [ ]:
# FETCH DATA FOR ACCOUNT 9913 TO CHECK DATA POPLULATION
print(conn.gsql('''
INTERPRET QUERY () FOR GRAPH AMLSim { 
  seed = {Account.*};
  S1 = SELECT s FROM seed:s WHERE s.id == "9913"; 
  PRINT S1; 
}
''', options=[]))


## Queries for GraphStudio
Select Account Query
```
CREATE QUERY selectAccount(STRING acct) FOR GRAPH AMLSim { 
  seed = {Account.*};
  S1 = SELECT s FROM seed:s WHERE s.id == acct; 
  PRINT S1; 
}
```
Select Account Query
```
CREATE QUERY selectTopPageRank() FOR GRAPH AMLSim { 
  seed = {Account.*};
  S1 = SELECT s FROM seed:s ORDER BY s.pagerank DESC LIMIT 10;
  PRINT S1; 
}
```
Select Account Transactions
```
CREATE QUERY selectAccountTx(STRING acct) FOR GRAPH AMLSim { 
  ListAccum<EDGE> @@txSend, @@txRecieve;
  seed = {Account.*};
  
  SendTx = SELECT tgt FROM seed:s -(Send_Transaction:e)-> Transaction:tgt
           WHERE s.id == acct
           ACCUM @@txSend +=  e;
  
  RecieveTx = SELECT tgt FROM seed:s -(reverse_Recieve_Transaction:e)-> Transaction:tgt
              WHERE s.id == acct
              ACCUM @@txRecieve += e;
  
  PRINT @@txSend, @@txRecieve; 
}
```

In [ ]:
# CREATE/INSTALL QUERY FOR ACCOUNT INFO
print(conn.gsql('''
CREATE QUERY selectAccount(STRING acct) FOR GRAPH AMLSim { 
  seed = {Account.*};
  S1 = SELECT s FROM seed:s WHERE s.id == acct; 
  PRINT S1; 
}
INSTALL QUERY selectAccount
''', options=[]))

In [ ]:
# RUN INSTALLED QUERY selectAccount
print(conn.runInstalledQuery("selectAccount", {"acct": "9913"}))

In [ ]:
# CREATE/INSTALL QUERY FOR ACCOUNTS TOP PAGERANK SCORES
print(conn.gsql('''
CREATE QUERY selectTopPageRank() FOR GRAPH AMLSim { 
  seed = {Account.*};
  S1 = SELECT s FROM seed:s ORDER BY s.pagerank DESC LIMIT 10;
  PRINT S1; 
}
INSTALL QUERY selectTopPageRank
''', options=[]))

In [ ]:
# RUN INSTALLED QUERY selectAccount
print(conn.runInstalledQuery("selectTopPageRank", {}))

In [ ]:
# CREATE/INSTALL QUERY FOR ACCOUNTS TRANSACTIONS
print(conn.gsql('''
CREATE QUERY selectAccountTx(STRING acct) FOR GRAPH AMLSim { 
  ListAccum<EDGE> @@txSend, @@txRecieve;
  seed = {Account.*};
  
  SendTx = SELECT tgt FROM seed:s -(Send_Transaction:e)-> Transaction:tgt
           WHERE s.id == acct
           ACCUM @@txSend +=  e;
  
  RecieveTx = SELECT tgt FROM seed:s -(reverse_Recieve_Transaction:e)-> Transaction:tgt
              WHERE s.id == acct
              ACCUM @@txRecieve += e;
  
  PRINT @@txSend, @@txRecieve; 
}
''', options=[]))

In [ ]:
# RUN INSTALLED QUERY selectAccountTx
print(conn.runInstalledQuery("selectAccountTx", {"acct": "9913"}))

# WARNING: DROP ALL - Will Delete everything in your graph!
Use this if you would like to start the notebook lab from the beginning.

In [ ]:
#print(conn.gsql('''DROP ALL''', options=[]))